# MVP-L: 테스트 및 평가

이 노트북은 KITTI 3D 평가 메트릭을 사용하여 모델 성능을 평가합니다.

## 목표
- KITTI 3D 평가 메트릭 (3D AP 등)
- 성능 분석 및 리포트 생성

In [ ]:
# Cell 1: 라이브러리 임포트
import numpy as np
from pathlib import Path
from collections import defaultdict

print("평가 모듈 준비 완료")

In [ ]:
# Cell 2: 3D IoU 계산
def compute_3d_iou(box1, box2):
    """
    3D 바운딩 박스 IoU 계산
    
    Args:
        box1, box2: dict with keys 'center', 'size', 'rotation'
    
    Returns:
        iou: 3D IoU 값
    """
    # 간단한 구현 (실제로는 더 복잡한 3D intersection 계산 필요)
    center1 = np.array(box1['center'])
    size1 = np.array(box1['size'])
    center2 = np.array(box2['center'])
    size2 = np.array(box2['size'])
    
    # 거리 기반 간단한 IoU 근사
    distance = np.linalg.norm(center1 - center2)
    volume1 = np.prod(size1)
    volume2 = np.prod(size2)
    
    # 거리가 너무 멀면 IoU = 0
    max_distance = np.linalg.norm(size1) + np.linalg.norm(size2)
    if distance > max_distance:
        return 0.0
    
    # 간단한 IoU 근사
    overlap_ratio = 1.0 - (distance / max_distance)
    min_volume = min(volume1, volume2)
    intersection = min_volume * overlap_ratio
    union = volume1 + volume2 - intersection
    
    return intersection / union if union > 0 else 0.0

print("3D IoU 계산 함수 정의 완료")

In [ ]:
# Cell 3: KITTI 3D 평가 (AP 계산)
def evaluate_3d_detections(gt_boxes, pred_boxes, iou_threshold=0.7):
    """
    3D 탐지 결과 평가
    
    Args:
        gt_boxes: list of dict with keys 'center', 'size', 'rotation', 'class'
        pred_boxes: list of dict with keys 'center', 'size', 'rotation', 'class', 'confidence'
        iou_threshold: 3D IoU 임계값
    
    Returns:
        results: 클래스별 AP 및 전체 mAP
    """
    # 클래스별로 분리
    classes = set([box.get('class', 'Car') for box in gt_boxes] + 
                 [box.get('class', 'Car') for box in pred_boxes])
    results = {}
    
    for cls in classes:
        gt_cls = [box for box in gt_boxes if box.get('class', 'Car') == cls]
        pred_cls = sorted([box for box in pred_boxes if box.get('class', 'Car') == cls],
                         key=lambda x: x.get('confidence', 0), reverse=True)
        
        tp = np.zeros(len(pred_cls))
        fp = np.zeros(len(pred_cls))
        gt_matched = [False] * len(gt_cls)
        
        # 각 예측에 대해 매칭
        for i, pred in enumerate(pred_cls):
            best_iou = 0
            best_gt_idx = -1
            
            for j, gt in enumerate(gt_cls):
                if gt_matched[j]:
                    continue
                
                iou = compute_3d_iou(pred, gt)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j
            
            if best_iou >= iou_threshold:
                tp[i] = 1
                gt_matched[best_gt_idx] = True
            else:
                fp[i] = 1
        
        # Precision-Recall 계산
        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)
        
        recalls = tp_cumsum / len(gt_cls) if len(gt_cls) > 0 else np.zeros(len(pred_cls))
        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
        
        # AP 계산 (11-point interpolation)
        ap = 0.0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11.0
        
        results[cls] = {'ap': ap}
    
    # mAP 계산
    map_score = np.mean([results[cls]['ap'] for cls in results])
    results['mAP'] = map_score
    
    return results

print("3D 평가 함수 정의 완료")